# dataBuilder.jl
Author: Tristan Smith
Start Date: 12.09.20
Last updated: 12.11.20

## Data needed: 
1.	Historical opening day game data
2.	Updated roster joined with the player’s 2019-2020 season stats
3.  Predictions for rookie MPG
    
## Steps:
1. Import Sports Reference API through PyCall
2. Form two dataframes: previous season team data and opening day results
3. Connect the dataframes as follows: Opening Day Team A...Team Data (year n-1) vs. Opening Team B...Team Data (year n-1)

In [1]:
using Pkg
Pkg.installed()
# Must have... (place Pkg.add() down below)

┌ Warning: Pkg.installed() is deprecated
└ @ Pkg /Users/julia/buildbot/worker/package_macos64/build/usr/share/julia/stdlib/v1.5/Pkg/src/Pkg.jl:554


Dict{String,VersionNumber} with 19 entries:
  "CSV"            => v"0.8.2"
  "StatsModels"    => v"0.6.15"
  "CSVFiles"       => v"1.0.0"
  "KeyedFrames"    => v"1.2.1"
  "JuMP"           => v"0.21.3"
  "TableReader"    => v"0.4.0"
  "Lathe"          => v"0.1.3"
  "ScikitLearn"    => v"0.6.3"
  "Queryverse"     => v"0.6.2"
  "PyCall"         => v"1.92.1"
  "DataFramesMeta" => v"0.6.0"
  "StatsBase"      => v"0.33.2"
  "IJulia"         => v"1.23.1"
  "Plots"          => v"1.9.1"
  "Feather"        => v"0.5.7"
  "Gadfly"         => v"1.3.1"
  "DataFrames"     => v"0.21.8"
  "Pandas"         => v"1.4.0"
  "GLM"            => v"1.3.11"

In [2]:
using CSV,DataFrames
results = CSV.read("nbaResults.csv", DataFrame)
teamStats = CSV.read("activeNBATeamStats.csv", DataFrame)
playerStats = CSV.read("playerStats.csv", DataFrame)
rosterStats = CSV.read("rosterStats.csv", DataFrame)
head(rosterStats)

,Column1,Team,Season,impliedFGM,impliedFGA,impliedFG%,implied3FGM,implied3FGA
,Int64,String,Int64,Int64,Int64,Float64,Int64,Int64
1,1,ANA,1967,343,758,0.452507,1,7
2,2,AND,1949,1483,4632,0.320164,0,0
3,3,ATL,1968,3605,7844,0.459587,0,0
4,4,ATL,1969,3674,7616,0.482405,0,0
5,5,ATL,1970,517,1126,0.459147,0,0
6,6,ATL,1981,2844,5999,0.474079,22,94


In [3]:
# Filter for years after NBA Merger, edit other dataframes and then
using DataFrames, DataFramesMeta
start_day = 1
start_month = 10
start_year1 = 1990
start_year2 = 1989
results = @where(results, :Y .>= start_year1)
teamStats = @where(teamStats, :Y .> start_year2)
playerStats = @where(playerStats, :Season .> start_year2)
rosterStats = @where(rosterStats, :Season .> start_year2)
head(teamStats)

,League,Y,Franchise,Team,G,FGM,FGA,FG%,3FGM
,String,Int64,String,String,Int64,Int64,Int64,Float64,String
1,NBA,1990,Atlanta Hawks,ATL,82,3349,7223,0.46,271
2,NBA,1990,Boston Celtics,BOS,82,3695,7214,0.51,109
3,NBA,1990,Brooklyn Nets,NJN,82,3311,7459,0.44,161
4,NBA,1990,Charlotte Hornets,CHA,82,3286,7033,0.47,131
5,NBA,1990,Chicago Bulls,CHI,82,3632,7125,0.51,155
6,NBA,1990,Cleveland Cavaliers,CLE,82,3259,6857,0.48,160


In [4]:
head(rosterStats)

,Column1,Team,Season,impliedFGM,impliedFGA,impliedFG%,implied3FGM,implied3FGA
,Int64,String,Int64,Int64,Int64,Float64,Int64,Int64
1,15,ATL,1990,3347,7220,0.463573,271,836
2,16,ATL,1991,3397,7265,0.467584,196,633
3,17,ATL,1992,3293,7028,0.468554,367,1020
4,18,ATL,1993,2542,5453,0.466165,206,629
5,19,ATL,1994,2156,4848,0.444719,392,1139
6,20,ATL,1995,2358,5334,0.44207,548,1525


In [5]:
fullTeamDF = join(teamStats, rosterStats, on = [:Team, :Y => :Season], kind = :left)

,League,Y,Franchise,Team,G,FGM,FGA,FG%,3FGM
,String,Int64,String,String,Int64,Int64,Int64,Float64,String
1,NBA,1990,Atlanta Hawks,ATL,82,3349,7223,0.46,271
2,NBA,1990,Boston Celtics,BOS,82,3695,7214,0.51,109
3,NBA,1990,Brooklyn Nets,NJN,82,3311,7459,0.44,161
4,NBA,1990,Charlotte Hornets,CHA,82,3286,7033,0.47,131
5,NBA,1990,Chicago Bulls,CHI,82,3632,7125,0.51,155
6,NBA,1990,Cleveland Cavaliers,CLE,82,3259,6857,0.48,160
7,NBA,1990,Dallas Mavericks,DAL,82,3245,6890,0.47,193
8,NBA,1990,Denver Nuggets,DEN,82,3901,8868,0.44,300
9,NBA,1990,Detroit Pistons,DET,82,3194,6875,0.46,131


In [8]:
# Double check CSV
CSV.write("fullTeamDF.csv", fullTeamDF) # No missing values with new years

"fullTeamDF.csv"

In [6]:
describe(fullTeamDF)

,variable,mean,min,median,max,nunique,nmissing
,Symbol,Union…,Any,Union…,Any,Union…,Union…
1,League,,NBA,,NBA,1,
2,Y,2004.77,1990,2005.0,2019,,
3,Franchise,,Atlanta Hawks,,Washington Wizards,30,
4,Team,,ATL,,WAS,34,
5,G,80.0023,50,82.0,82,,
6,FGM,3010.53,1539,3041.5,3901,,
7,FGA,6606.05,3561,6698.5,8868,,
8,FG%,0.455342,0.4,0.45,0.51,,
9,3FGM,,1007,,995,540,


In [ ]:
# Build difference dataframe
